In [1]:
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras import layers, models


In [3]:
# --- Fire Dataset Preprocessing ---
fire_dir = './../archive (6)/fire_dataset/fire_images'
non_fire_dir = './../archive (6)/fire_dataset/non_fire_images'

fire_images = [os.path.join(fire_dir, img) for img in os.listdir(fire_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]
non_fire_images = [os.path.join(non_fire_dir, img) for img in os.listdir(non_fire_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]

fire_labels = np.ones(len(fire_images), dtype=np.int32)
non_fire_labels = np.zeros(len(non_fire_images), dtype=np.int32)

all_fire_images = fire_images + non_fire_images
all_fire_labels = np.concatenate([fire_labels, non_fire_labels])

# Split fire dataset
fire_train_images, fire_val_images, fire_train_labels, fire_val_labels = train_test_split(
    all_fire_images, all_fire_labels, test_size=0.2, random_state=42
)

# --- Road Damage Dataset Preprocessing ---


In [4]:
road_dir = './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset'
road_data = []

for road_folder in os.listdir(road_dir):
    road_path = os.path.join(road_dir, road_folder)
    if os.path.isdir(road_path):
        raw_image_path = None
        lane_mask_path = None
        pothole_mask_path = None
        crack_mask_path = None

        for file in os.listdir(road_path):
            if 'raw' in file.lower():
                raw_image_path = os.path.join(road_path, file)
            elif 'lane' in file.lower():
                lane_mask_path = os.path.join(road_path, file)
            elif 'pothole' in file.lower():
                pothole_mask_path = os.path.join(road_path, file)
            elif 'crack' in file.lower():
                crack_mask_path = os.path.join(road_path, file)

        if raw_image_path and lane_mask_path and pothole_mask_path and crack_mask_path:
            road_data.append({
                'raw_image': raw_image_path,
                'lane_mask': lane_mask_path,
                'pothole_mask': pothole_mask_path,
                'crack_mask': crack_mask_path
            })

# Split road dataset
road_train_data, road_val_data = train_test_split(road_data, test_size=0.2, random_state=42)

# --- Data Loading and Preprocessing Functions ---


In [6]:
road_train_data

[{'raw_image': './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1093102_DF_251_251BDF0052_01123\\1093102_DF_251_251BDF0052_01123_RAW.jpg',
  'lane_mask': './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1093102_DF_251_251BDF0052_01123\\1093102_DF_251_251BDF0052_01123_LANE.png',
  'pothole_mask': './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1093102_DF_251_251BDF0052_01123\\1093102_DF_251_251BDF0052_01123_POTHOLE.png',
  'crack_mask': './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1093102_DF_251_251BDF0052_01123\\1093102_DF_251_251BDF0052_01123_CRACK.png'},
 {'raw_image': './../biankatpas-Cracks-and-Potholes-in-Road-Images-Datas

In [10]:
IMG_SIZE = (256, 256) # Define your desired image size
NUM_DAMAGE_CLASSES = 3 # lane, pothole, crack

def load_and_preprocess_fire_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3) # Or decode_png if your images are PNG
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32) # Normalize to [0, 1]
    return img, label

def load_and_preprocess_road_data(data_item):
    raw_img = tf.io.read_file(data_item['raw_image'])
    raw_img = tf.image.decode_jpeg(raw_img, channels=3)
    raw_img = tf.image.resize(raw_img, IMG_SIZE)
    raw_img = tf.image.convert_image_dtype(raw_img, tf.float32)

    lane_mask = tf.io.read_file(data_item['lane_mask'])
    lane_mask = tf.image.decode_png(lane_mask, channels=1) # Assuming grayscale masks
    lane_mask = tf.image.resize(lane_mask, IMG_SIZE, method='nearest')
    lane_mask = tf.cast(lane_mask > 128, tf.int32) # Convert to binary (0 or 1)

    pothole_mask = tf.io.read_file(data_item['pothole_mask'])
    pothole_mask = tf.image.decode_png(pothole_mask, channels=1)
    pothole_mask = tf.image.resize(pothole_mask, IMG_SIZE, method='nearest')
    pothole_mask = tf.cast(pothole_mask > 128, tf.int32)

    crack_mask = tf.io.read_file(data_item['crack_mask'])
    crack_mask = tf.image.decode_png(crack_mask, channels=1)
    crack_mask = tf.image.resize(crack_mask, IMG_SIZE, method='nearest')
    crack_mask = tf.cast(crack_mask > 128, tf.int32)

    # Combine masks into a multi-channel mask (you might need to adjust this based on your loss function)
    combined_mask = tf.concat([lane_mask, pothole_mask, crack_mask], axis=-1)
    combined_mask = tf.cast(combined_mask, tf.float32) # Or tf.int32 depending on your loss

    return raw_img, combined_mask



In [11]:
# --- Create TensorFlow Datasets ---
# Fire Training Dataset
fire_train_dataset = tf.data.Dataset.from_tensor_slices((fire_train_images, fire_train_labels))
fire_train_dataset = fire_train_dataset.map(load_and_preprocess_fire_image).batch(32).prefetch(tf.data.AUTOTUNE)

# Fire Validation Dataset
fire_val_dataset = tf.data.Dataset.from_tensor_slices((fire_val_images, fire_val_labels))
fire_val_dataset = fire_val_dataset.map(load_and_preprocess_fire_image).batch(32).prefetch(tf.data.AUTOTUNE)

# # Road Training Dataset
# road_train_dataset = tf.data.Dataset.from_tensor_slices(road_train_data)
# road_train_dataset = road_train_dataset.map(load_and_preprocess_road_data).batch(32).prefetch(tf.data.AUTOTUNE)

# # Road Validation Dataset
# road_val_dataset = tf.data.Dataset.from_tensor_slices(road_val_data)
# road_val_dataset = road_val_dataset.map(load_and_preprocess_road_data).batch(32).prefetch(tf.data.AUTOTUNE)

# --- Combining Datasets for Multi-Task Training ---
# You'll need a strategy to train on both datasets. One common approach is to
# interleave the datasets or train in separate epochs.

# Example of interleaving (you might need to adjust the sampling):
# combined_train_dataset = tf.data.Dataset.sample_from_datasets(
#     [fire_train_dataset.map(lambda img, label: (img, {'fire_output': label, 'segmentation_output': tf.zeros_like(img[..., :NUM_DAMAGE_CLASSES]))}),
#      road_train_dataset.map(lambda img, mask: (img, {'fire_output': tf.constant([0.5]), 'segmentation_output': mask}))], # Adjust fire label for road data
#     weights=[0.5, 0.5] # Adjust weights based on dataset sizes
# ).batch(32).prefetch(tf.data.AUTOTUNE)

# A simpler approach might be to train in epochs or alternate batches and handle
# the different output structures in your training loop.

# For validation, you would likely evaluate the fire and road damage tasks separately
# using their respective validation datasets.

In [16]:
road_dir = './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset'
raw_image_paths = []
lane_mask_paths = []
pothole_mask_paths = []
crack_mask_paths = []

for road_folder in os.listdir(road_dir):
    road_path = os.path.join(road_dir, road_folder)
    if os.path.isdir(road_path):
        raw_image_path = None
        lane_mask_path = None
        pothole_mask_path = None
        crack_mask_path = None

        for file in os.listdir(road_path):
            if 'raw' in file.lower():
                raw_image_path = os.path.join(road_path, file)
            elif 'lane' in file.lower():
                lane_mask_path = os.path.join(road_path, file)
            elif 'pothole' in file.lower():
                pothole_mask_path = os.path.join(road_path, file)
            elif 'crack' in file.lower():
                crack_mask_path = os.path.join(road_path, file)

        if raw_image_path and lane_mask_path and pothole_mask_path and crack_mask_path:
            raw_image_paths.append(raw_image_path)
            lane_mask_paths.append(lane_mask_path)
            pothole_mask_paths.append(pothole_mask_path)
            crack_mask_paths.append(crack_mask_path)

# Combine paths
road_image_mask_paths = list(zip(raw_image_paths, lane_mask_paths, pothole_mask_paths, crack_mask_paths))
print(road_image_mask_paths)
# Split road dataset (splitting lists of paths)
road_train_paths, road_val_paths = train_test_split(road_image_mask_paths, test_size=0.2, random_state=42)

# --- Data Loading and Preprocessing Functions (Revised) ---
IMG_SIZE = (256, 256)
NUM_DAMAGE_CLASSES = 3

def load_and_preprocess_road_data(raw_path, lane_path, pothole_path, crack_path):
    raw_img = tf.io.read_file(raw_path)
    raw_img = tf.image.decode_jpeg(raw_img, channels=3)
    raw_img = tf.image.resize(raw_img, IMG_SIZE)
    raw_img = tf.image.convert_image_dtype(raw_img, tf.float32)

    lane_mask = tf.io.read_file(lane_path)
    lane_mask = tf.image.decode_png(lane_mask, channels=1)
    lane_mask = tf.image.resize(lane_mask, IMG_SIZE, method='nearest')
    lane_mask = tf.cast(lane_mask > 128, tf.int32)

    pothole_mask = tf.io.read_file(pothole_path)
    pothole_mask = tf.image.decode_png(pothole_mask, channels=1)
    pothole_mask = tf.image.resize(pothole_mask, IMG_SIZE, method='nearest')
    pothole_mask = tf.cast(pothole_mask > 128, tf.int32)

    crack_mask = tf.io.read_file(crack_path)
    crack_mask = tf.image.decode_png(crack_mask, channels=1)
    crack_mask = tf.image.resize(crack_mask, IMG_SIZE, method='nearest')
    crack_mask = tf.cast(crack_mask > 128, tf.int32)

    combined_mask = tf.stack([lane_mask[..., 0], pothole_mask[..., 0], crack_mask[..., 0]], axis=-1)
    combined_mask = tf.cast(combined_mask, tf.float32)

    return raw_img, combined_mask

# --- Create TensorFlow Datasets (Revised) ---
# Road Training Dataset
# road_train_dataset = tf.data.Dataset.from_tensor_slices(road_train_paths)
# road_val_dataset = road_train_dataset.map(lambda raw_path, lane_path, pothole_path, crack_path: load_and_preprocess_road_data(raw_path, lane_path, pothole_path, crack_path)).batch(32).prefetch(tf.data.AUTOTUNE)


# # Road Validation Dataset
# road_val_dataset = tf.data.Dataset.from_tensor_slices(road_val_paths)
# road_val_dataset = road_val_dataset = road_val_dataset.map(lambda raw_path, lane_path, pothole_path, crack_path: load_and_preprocess_road_data(raw_path, lane_path, pothole_path, crack_path)).batch(32).prefetch(tf.data.AUTOTUNE)
tf.config.run_functions_eagerly(True)
# Remember to turn it back off if needed
# road_train_dataset = tf.data.Dataset.from_tensor_slices(road_train_paths)
raw_paths, lane_paths, pothole_paths, crack_paths = zip(*road_train_paths)

train_ds = tf.data.Dataset.from_tensor_slices((list(raw_paths), list(lane_paths), list(pothole_paths), list(crack_paths)))
val_raw, val_lane, val_pothole, val_crack = zip(*road_val_paths)
val_ds = tf.data.Dataset.from_tensor_slices((list(val_raw), list(val_lane), list(val_pothole), list(val_crack)))

road_train_dataset = train_ds.map(load_and_preprocess_road_data).batch(32).prefetch(tf.data.AUTOTUNE)
road_val_dataset = val_ds.map(load_and_preprocess_road_data).batch(32).prefetch(tf.data.AUTOTUNE)


# road_train_dataset = road_train_dataset.map(lambda paths: load_and_preprocess_road_data(*paths)).batch(32).prefetch(tf.data.AUTOTUNE)
road_train_dataset = road_train_dataset.map(
    lambda image_path, mask_path: load_and_preprocess_road_data(image_path, mask_path)
).batch(32).prefetch(tf.data.AUTOTUNE)


# Road Validation Dataset
road_val_dataset = tf.data.Dataset.from_tensor_slices(road_val_paths)
road_val_dataset = road_val_dataset.map(lambda paths: load_and_preprocess_road_data(*paths)).batch(32).prefetch(tf.data.AUTOTUNE)


# ... your dataset creation code ...
tf.config.run_functions_eagerly(False) 


[('./../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1007599_RS_386_386RS289112_28920\\1007599_RS_386_386RS289112_28920_RAW.jpg', './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1007599_RS_386_386RS289112_28920\\1007599_RS_386_386RS289112_28920_LANE.png', './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1007599_RS_386_386RS289112_28920\\1007599_RS_386_386RS289112_28920_POTHOLE.png', './../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset\\1007599_RS_386_386RS289112_28920\\1007599_RS_386_386RS289112_28920_CRACK.png'), ('./../biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f2

TypeError: in user code:

    File "C:\Users\saich\AppData\Local\Temp\ipykernel_29504\491925026.py", line 91, in None  *
        lambda image_path, mask_path: load_and_preprocess_road_data(image_path, mask_path)

    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__load_and_preprocess_road_data() missing 2 required positional arguments: 'pothole_path' and 'crack_path'


In [ ]:
def custom_segmentation_head(encoder_output, num_classes):
    up1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(encoder_output)
    conv_up1 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(up1)
    conv_up1 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(conv_up1)

    up2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv_up1)
    conv_up2 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(up2)
    conv_up2 = layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv_up2) # Output segmentation map

    return conv_up2

In [ ]:
def build_custom_multi_task_model(input_shape, num_damage_classes=3, num_filters_base=32):
    input_tensor = layers.Input(shape=input_shape)
    encoder_output = custom_encoder(input_tensor, num_filters_base)

    # Fire Classification Head
    fire_head = layers.Flatten()(encoder_output)
    fire_head = layers.Dense(128, activation='relu')(fire_head)
    fire_output = layers.Dense(1, activation='sigmoid', name='fire_output')(fire_head)

    # Road Damage Segmentation Head
    segmentation_output = custom_segmentation_head(encoder_output, num_damage_classes)

    model = models.Model(inputs=input_tensor, outputs=[fire_output, segmentation_output])
    return model

# Example usage:
input_shape = (256, 256, 3)
num_damage_classes = 3
custom_model = build_custom_multi_task_model(input_shape, num_damage_classes)
custom_model.compile(
    optimizer='adam',
    loss={'fire_output': 'binary_crossentropy', 'segmentation_output': 'categorical_crossentropy'}, # Or other segmentation losses like Dice
    loss_weights={'fire_output': 1.0, 'segmentation_output': 1.0}, # Adjust weights as needed
    metrics={'fire_output': 'accuracy', 'segmentation_output': ['mean_iou']} # Add relevant segmentation metrics
)


In [ ]:
import tensorflow as tf

def train_model(model, fire_train_dataset, road_train_dataset, fire_val_dataset, road_val_dataset, epochs):
    optimizer = model.optimizer # Get the optimizer from the compiled model
    fire_loss_fn = tf.keras.losses.BinaryCrossentropy()
    segmentation_loss_fn = tf.keras.losses.CategoricalCrossentropy() # Or your chosen segmentation loss

    def train_step(images, fire_labels, segmentation_masks):
        with tf.GradientTape() as tape:
            fire_predictions, segmentation_predictions = model(images, training=True)
            fire_loss = fire_loss_fn(fire_labels, fire_predictions)
            segmentation_loss = segmentation_loss_fn(segmentation_masks, segmentation_predictions)
            total_loss = model.loss_weights['fire_output'] * fire_loss + model.loss_weights['segmentation_output'] * segmentation_loss
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return fire_loss, segmentation_loss

    def val_step(images, fire_labels, segmentation_masks):
        fire_predictions, segmentation_predictions = model(images, training=False)
        fire_loss = fire_loss_fn(fire_labels, fire_predictions)
        segmentation_loss = segmentation_loss_fn(segmentation_masks, segmentation_predictions)
        return fire_loss, segmentation_loss

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}")
        fire_train_loss_avg = tf.keras.metrics.Mean()
        segmentation_train_loss_avg = tf.keras.metrics.Mean()
        fire_val_loss_avg = tf.keras.metrics.Mean()
        segmentation_val_loss_avg = tf.keras.metrics.Mean()
        fire_train_accuracy_metric = tf.keras.metrics.BinaryAccuracy()
        segmentation_train_iou_metric = tf.keras.metrics.MeanIoU(num_classes=NUM_DAMAGE_CLASSES)
        fire_val_accuracy_metric = tf.keras.metrics.BinaryAccuracy()
        segmentation_val_iou_metric = tf.keras.metrics.MeanIoU(num_classes=NUM_DAMAGE_CLASSES)

        # --- Training on Fire Dataset ---
        print("Training on Fire Dataset...")
        for images, labels in fire_train_dataset:
            # Create dummy segmentation masks of the correct shape but filled with zeros
            dummy_segmentation_masks = tf.zeros(shape=(tf.shape(images)[0], IMG_SIZE[0], IMG_SIZE[1], NUM_DAMAGE_CLASSES), dtype=tf.float32)
            f_loss, _ = train_step(images, labels[:, tf.newaxis], dummy_segmentation_masks) # Reshape labels for binary_crossentropy
            fire_train_loss_avg.update_state(f_loss)
            fire_train_accuracy_metric.update_state(labels, tf.round(model(images)[0]))

        print(f"  Fire Train Loss: {fire_train_loss_avg.result():.4f}, Fire Train Accuracy: {fire_train_accuracy_metric.result():.4f}")

        # --- Validation on Fire Dataset ---
        print("Validating on Fire Dataset...")
        for images, labels in fire_val_dataset:
            dummy_segmentation_masks = tf.zeros(shape=(tf.shape(images)[0], IMG_SIZE[0], IMG_SIZE[1], NUM_DAMAGE_CLASSES), dtype=tf.float32)
            f_loss, _ = val_step(images, labels[:, tf.newaxis], dummy_segmentation_masks)
            fire_val_loss_avg.update_state(f_loss)
            fire_val_accuracy_metric.update_state(labels, tf.round(model(images)[0]))

        print(f"  Fire Val Loss: {fire_val_loss_avg.result():.4f}, Fire Val Accuracy: {fire_val_accuracy_metric.result():.4f}")

        # --- Training on Road Damage Dataset ---
        print("Training on Road Damage Dataset...")
        for images, masks in road_train_dataset:
            # Create dummy fire labels (e.g., all zeros or a mix - the model should learn to ignore this for the segmentation task)
            dummy_fire_labels = tf.zeros(shape=(tf.shape(images)[0], 1), dtype=tf.float32)
            _, s_loss = train_step(images, dummy_fire_labels, masks)
            segmentation_train_loss_avg.update_state(s_loss)
            segmentation_predictions = model(images)[1]
            segmentation_train_iou_metric.update_state(tf.argmax(masks, axis=-1), tf.argmax(segmentation_predictions, axis=-1))

        print(f"  Segmentation Train Loss: {segmentation_train_loss_avg.result():.4f}, Segmentation Train IoU: {segmentation_train_iou_metric.result():.4f}")

        # --- Validation on Road Damage Dataset ---
        print("Validating on Road Damage Dataset...")
        for images, masks in road_val_dataset:
            dummy_fire_labels = tf.zeros(shape=(tf.shape(images)[0], 1), dtype=tf.float32)
            _, s_loss = val_step(images, dummy_fire_labels, masks)
            segmentation_val_loss_avg.update_state(s_loss)
            segmentation_predictions = model(images)[1]
            segmentation_val_iou_metric.update_state(tf.argmax(masks, axis=-1), tf.argmax(segmentation_predictions, axis=-1))

        print(f"  Segmentation Val Loss: {segmentation_val_loss_avg.result():.4f}, Segmentation Val IoU: {segmentation_val_iou_metric.result():.4f}")
        print("-" * 30)

# Example usage:
epochs = 10 # Adjust the number of epochs
train_model(custom_model, fire_train_dataset, road_train_dataset, fire_val_dataset, road_val_dataset, epochs)

In [1]:
pip install torch torchvision


^C
Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🔁 Data Transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # normalize to [-1, 1]
])

# 📁 Load Dataset
dataset = ImageFolder(root="./../archive (6)/fire_dataset", transform=transform)

# 🔀 Train/Validation Split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 🧠 Define CNN Model
class FireCNN(nn.Module):
    def __init__(self):
        super(FireCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: fire, non-fire

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 64, 64]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 32, 32]
        x = x.view(-1, 32 * 32 * 32)          # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = FireCNN().to(device)

# 🧮 Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 🏋️ Training Loop
def train_model(num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        val_acc = evaluate_model()
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {acc:.2f}%, Val Acc: {val_acc:.2f}%")

    # 💾 Save the model
    torch.save(model.state_dict(), "fire_cnn.pth")
    print("Model saved as fire_cnn.pth")

# ✅ Evaluation on Validation Set
def evaluate_model():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100 * correct / total

# 🧪 Run Training
if __name__ == "__main__":
    train_model(num_epochs=10)


ModuleNotFoundError: No module named 'torch'